<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Sentiment/merge_clarifai_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Crea il csv contenente il merge tra 
# -csv contenente tutte le recensioni del ristorante
# -csv contenente gli user presi random da Foto_User

#da cambiare:
#nome_ristorante
#nome_ristorante_staccato
#users_restaurant_id (link condivisibile del csv con tutte le review)
#file_list (cartella da cui prendo le foto)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante = 'TupeloHoney'
nome_ristorante_staccato = 'Tupelo Honey'

# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance) > (nomeRistorante)_relevance_desc.csv)
#File dove ci sono id e recensione
users_restaurant_id = '1aOTHWSNdkjzW7EyyaomdXxgnzjXdqN7S'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nome_ristorante_staccato + '_relevance_desc.csv')

#leggo il file csv del ristorante
restaurant = pd.read_csv('%s' % nome_ristorante_staccato + '_relevance_desc.csv')

#data_frame contenente gli utenti di Foto_Users
df = pd.DataFrame(columns = ['id'])

#lista con gli utenti 
#percorso cartella: data > face_data > Foto_User > nome_ristorante_10%Random
file_list = drive.ListFile({'q': "'1Y4kVSFpVyLUA6uIAE9pAN12U82KNRZ6-' in parents and trashed=false"}).GetList()    #cartella da cui prendo le foto

#scrivo gli utenti nel csv
cnt = 0
for file1 in file_list:
  print(file1['title'][0:22])
  df.loc[cnt, 'id'] = '%s' % file1['title'][0:22]
  cnt = cnt + 1
print(df)

#foto = pd.read_csv('%s' % nome_ristorante + '_FotoRandom.csv')
merged = df.merge(restaurant,how = 'left',left_on='id', right_on='user_id')

review_random = merged.drop(columns=['id'])

#csv contenente user_id e review
review_random.to_csv('%s' % nome_ristorante + '_merge.csv')

folder_id = '1qd07afmJktKju4tsxYfanJWEafV3JSEb'    #cartella dove metto il csv
file_metadata = {'title': '%s' % nome_ristorante + '_merge.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
folder = drive.CreateFile(file_metadata)
folder.SetContentFile('%s' % nome_ristorante + '_merge.csv') #The contents of the file
folder.Upload()

eGCft04d3kw-jXI5XYE4UQ
ECL-p9PXfWaWaY6ocioSKw
9eiqEQzRrhPjvMDoGRq20Q
GWOf1oB2mTalRY_A-EjSiQ
1t9pHpZUokQCt8ihAG9JLg
fDKwVe0dHSKqaRPVULvZSg
SsfDStg0J2SPCbYXh8oS0g
VXQkBxGcrjnzlKi_x4mkRA
98nfKyfvmHUJ8U4PFh4WtQ
_iggvTE7DLpDFLS5DXLc2Q
tX-qZGlwhINYP-l2_WhG_w
8urfFt-_QlHYAihPK8p-TA
B5XDgCyNNsxpsJYy0YJbkQ
P5Xmi0JUMSorpi1S1TM0qQ
yCvQKqK2Bcint3r6YcD7VA
XatJ1Tkqw5dw5n7WeWr_FA
-URDhJAzkssf0AT2SPIB2Q
Pqz3OcVaI53zdlzSqqXL5A
FeaH3E-XDRHfrzV4Ea0LwQ
wCDC5NVQLKdTa-OwzLG9vg
SHO8XXRSeTE6_gLtyOc_JQ
Qz6jpfNMNPTiqSxrIVpbKg
Z1oe8fyaibWh7IudGA520A
JeF43VrWyHcqQIyLo0XqOw
0_XCo17kR9gwjvEi00Zygg
1EymEITWdhu_-xKcS2HbdQ
xSlZoZ1IDejYVgCGGgBbZQ
MFve2_inSIJOF6_-XtU0Ig
c0YdiGs0APd4AjspTHVdcQ
iFwBRXu3lR9OwSaw_NJxfA
9FhZNgkCVsBtFDHa-pRcDw
QxObX125jzv5IWZfVUm0SA
EXR9Avq2FC6TsTdrKwQHCw
TTsbBiTfAhvbyTtmvHwBYQ
Ji53C7mQHr3mPeTLyHpDPg
3HO8JhkD6F4xq4DfzzNfBA
lGBR8aOZso91UcEzZ-cAng
rAhSQ0NNLtClXTHJ-cydVQ
6Bt82-KbLZ272wr0bhhnow
5_vvuAY9sOVbJtBon8ce2A
mcuez7RRxuIyAu3ZT9zQCw
9cQpfD1hPQxRQrCREZsvkQ
olW3TJfjHQadUJ7XBZqfpg
TmLqu6R0D2X